### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

# to prevent figure 
# to show in the notebook
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

import plotting as plotting

### TODO: update this (GROUP OF 5 VERSION)

In [3]:
date = "24-12-16"
pathfile = f"Pulling data/{date}/groups_of_5/triple/"

### From here; automatic run

In [4]:
root = f"/Users/michaelmoret/Library/CloudStorage/GoogleDrive-michael@externa.bio/.shortcut-targets-by-id/1BdUNsBjDh5Gee_76jCiKB1C_CwG0ercP/{pathfile}"

MODE = "triple"
dev = False

# number of hairs from the same group together
n_together = 5
total_hairs = 100

In [5]:
if not root.endswith("/"):
    root += "/"

sep_category = "$" # separates categories like date, experiments, etc
sep_count = "%" # separated the number of hairs by experiment

In [6]:
all_dfs = []
all_fns = []

for file in os.listdir(root):
    if file.endswith(".txt"):
        print("\n*****************************")
        print(file)
        df = plotting.get_df_from_file(root + file,
                                       skip=2)
        # clean the df
        if MODE in file:
            df = plotting.clean_triple(df)
            assert "triple" in file
        else:
            raise ValueError(f"Not a {MODE} experiment")

        splitted_name = file.split(sep_category)
        # get the experiments; i.e. not the date
        # not the single or triple etc
        splitted_name = splitted_name[2:-1]
        print("splitted_name: ", splitted_name)
        # add the experiment name in the dataframe
        all_names = []
        
        # trick to get the right labels
        labelling_round = 0
        while labelling_round!=total_hairs:
            for entries in splitted_name:
                times_name = entries.split(sep_count)
                _name = times_name[1]
                all_names += [_name] * n_together
                labelling_round += n_together

        print(f"len df: {len(df)}, len names: {len(all_names)}")
        df["Name"] = all_names
        # remove nan
        df = df.dropna()
        all_dfs.append(df)
        all_fns.append(file.replace(".txt", ""))

print(all_names)


*****************************
20241213$IHNY$25%phosctrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$triple.txt
Missing records: []
splitted_name:  ['25%phosctrl', '25% PK30-2 0.01', '25%PK30-2  0.03', '25% PK30-2 0.05']
len df: 100, len names: 100

*****************************
20241216$IHNY$25%AActrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$triple.txt
Missing records: []
splitted_name:  ['25%AActrl', '25% PK30-2 0.01', '25%PK30-2  0.03', '25% PK30-2 0.05']
len df: 100, len names: 100
['AActrl', 'AActrl', 'AActrl', 'AActrl', 'AActrl', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', 'PK30-2  0.03', 'PK30-2  0.03', 'PK30-2  0.03', 'PK30-2  0.03', 'PK30-2  0.03', ' PK30-2 0.05', ' PK30-2 0.05', ' PK30-2 0.05', ' PK30-2 0.05', ' PK30-2 0.05', 'AActrl', 'AActrl', 'AActrl', 'AActrl', 'AActrl', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', ' PK30-2 0.01', 'PK30-2  0.03', 'PK30-2  0.03', 'PK30-2  0.03', 'PK30-2  0.03', 'PK30-2  0.03

In [7]:
def master_plots(df, header, ymin, ymax, savepath, show=False, verbose=False):
    fig, ax, removed = plotting.create_boxplot(df, header, ymin, ymax)
    #save removed data
    removed.to_csv(f"{savepath}{header}_removed.csv", index=True)
    if verbose:
        print(f"{len(removed)} removed data points for {header}")
    # plot
    fig.savefig(f"{savepath}{header}.png")
    if show: 
        plt.show()
        plt.close()

In [8]:
len(all_dfs), len(all_fns)

(2, 2)

In [9]:
all_fns

['20241213$IHNY$25%phosctrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$triple',
 '20241216$IHNY$25%AActrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$triple']

In [10]:
for df, name in zip(all_dfs, all_fns):
    print("\n************************************")
    print(f"{name}\n")
    print(f"df length: {len(df)}")
    try:
        savepath = f"{root}{name}/"
        if dev:
            savepath = f"{root}dev/"
        os.makedirs(savepath, exist_ok=True)
        
        try:
            header = 'MEAN DIAMETER'
            ymin = 20
            ymax = 120
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK STRESS'
            ymin = 120
            ymax = 280
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'TOUGHNESS'
            ymin = None
            ymax = None
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)
        
        try:
            header = 'ELASTIC GRADIENT'
            ymin = 0
            ymax =  140
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'ELASTIC EMOD'
            ymin = 2.5
            ymax =  6.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        # scatter plots
        y_col = 'ELASTIC EMOD'
        x_col = 'MEAN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'ELASTIC EMOD'
        x_col = 'MIN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'ELASTIC EMOD'
        x_col = 'MAX DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'MEAN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'MIN DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'MAX DIAMETER'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK STRESS'
        x_col = 'RECORD'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

    except:
        print(f"\nERROR with {name}\n")
        continue

print("\n\nplotting DONE")


************************************
20241213$IHNY$25%phosctrl$25% PK30-2 0.01$25%PK30-2  0.03$25% PK30-2 0.05$triple

df length: 77

Creating boxplot for MEAN DIAMETER
Removed outliers for phosctrl : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for  PK30-2 0.01 : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for PK30-2  0.03 : Series([], Name: MEAN DIAMETER, dtype: float64)
Removed outliers for  PK30-2 0.05 : 75    94.5
98    96.4
Name: MEAN DIAMETER, dtype: float64
Stats. significance:  PK30-2 0.01 and  PK30-2 0.05: p = 0.0190
Stats. significance: PK30-2  0.03 and  PK30-2 0.05: p = 0.0111
Median for phosctrl: 73.9
Median for  PK30-2 0.01: 76.9
Median for PK30-2  0.03: 76.8
Median for  PK30-2 0.05: 67.1

Creating boxplot for BREAK STRESS
Removed outliers for phosctrl : 80    158.86773
Name: BREAK STRESS, dtype: float64
Removed outliers for  PK30-2 0.01 : 7    128.467115
Name: BREAK STRESS, dtype: float64
Removed outliers for PK30-2  0.03 : 92   

/Users/michaelmoret/tinyverse/krap/notebooks_2/plotting.py:218: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=figsize)




plotting DONE
